In [5]:
#-----------------------------
# Use for Regression Equation, FStat, Pvalue


import pandas as pd
import numpy as np
import statsmodels.api as sm

# === Step 1: Load dataset ===
file_path = r"C:\Users\300393449\OneDrive - Douglas College\Documents\4th Semester\2_Business_Statistics_II\Python\DataScratch.xlsx"
sheet_name = "Reg_Model"

df = pd.read_excel(file_path, sheet_name=sheet_name)
print("Dataset:")
print(df)

# === Helpers ===
def clean_numeric_series(s: pd.Series) -> pd.Series:
    """Strip $, %, commas, spaces; coerce to numeric (NaN on failure)."""
    s = s.astype(str).str.replace(r'[\$,%]', '', regex=True).str.replace(',', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')

def coerce_all_numeric(X: pd.DataFrame) -> pd.DataFrame:
    out = X.copy()
    for c in out.columns:
        if out[c].dtype == 'object' or str(out[c].dtype).startswith('category'):
            out[c] = clean_numeric_series(out[c])
        else:
            # try safe numeric cast; if fails, becomes NaN
            out[c] = pd.to_numeric(out[c], errors='coerce')
    return out

def handle_categoricals(data: pd.DataFrame, custom_orders: dict | None = None):
    """Excel-style dummy encoding for any categorical predictors (exclude 'y')."""
    d = data.copy()
    custom_orders = custom_orders or {}
    cat_cols = [c for c in d.columns if c != 'y' and (d[c].dtype == 'object' or str(d[c].dtype).startswith('category'))]

    baseline_info = {}
    if not cat_cols:
        return d, baseline_info

    # enforce custom baselines if provided; else baseline = first alphabetical
    for col in cat_cols:
        if col in custom_orders:
            d[col] = pd.Categorical(d[col], categories=custom_orders[col], ordered=True)
            baseline_info[col] = custom_orders[col][0]
        else:
            vals = d[col].astype(str).str.strip()
            cats_sorted = sorted(vals.unique().tolist())
            d[col] = pd.Categorical(vals, categories=cats_sorted, ordered=True)
            baseline_info[col] = cats_sorted[0] if cats_sorted else None

    d = pd.get_dummies(d, columns=cat_cols, drop_first=True)
    return d, baseline_info

# === Step 2: Dummy-encode any categoricals (Excel-style) ===
custom_baselines = {}  # e.g., {'Month': ['June','July','August']}
df_dum, baseline_info = handle_categoricals(df, custom_baselines)

if baseline_info:
    print("\n[Dummy encoding summary] (baseline absorbed by intercept):")
    for col, base in baseline_info.items():
        created = [c for c in df_dum.columns if c.startswith(f"{col}_")]
        print(f" - {col}: baseline = '{base}', dummies = {created}")

# === Step 3: Build model matrices ===
if 'y' not in df_dum.columns:
    raise ValueError("Dataset must contain a 'y' column.")

y = df_dum['y']
X = df_dum.drop(columns=['y'])

# --- DIAGNOSTIC: show offending dtypes before coercion ---
non_numeric_cols = [c for c in X.columns if (X[c].dtype == 'object' or str(X[c].dtype).startswith('category'))]
if y.dtype == 'object' or str(y.dtype).startswith('category') or non_numeric_cols:
    print("\n[Info] Coercing non-numeric columns to numeric...")
    if y.dtype == 'object' or str(y.dtype).startswith('category'):
        print(" - y is non-numeric; cleaning it.")
    if non_numeric_cols:
        print(" - Non-numeric predictors:", non_numeric_cols)

# Coerce to numeric safely (currency, commas, percents -> handled)
y = clean_numeric_series(y)
X = coerce_all_numeric(X)

# Drop rows with missing y or any missing X
mask = y.notna() & X.notna().all(axis=1)
dropped_rows = int(len(y) - mask.sum())
if dropped_rows > 0:
    print(f"\n[Warning] Dropping {dropped_rows} row(s) with non-numeric or missing values after cleaning.")
y = y[mask]
X = X[mask]

# Drop predictors that are constant or entirely NaN
const_or_empty = [c for c in X.columns if X[c].nunique(dropna=True) <= 1]
if const_or_empty:
    print(f"[Info] Dropping constant/empty column(s): {const_or_empty}")
    X = X.drop(columns=const_or_empty)

# Final safety: ensure float dtype
X = X.astype(float)
y = y.astype(float)

# Add intercept
X = sm.add_constant(X, has_constant='add')

# === Step 4: Fit OLS ===
model = sm.OLS(y, X).fit()

# === Step 5: Regression equation ===
coef = model.params
eq_parts = [f"y = {coef['const']:.4f}"]
for col in X.columns:
    if col == 'const':
        continue
    eq_parts.append(f"+ ({coef[col]:.4f})*{col}")
equation = " ".join(eq_parts)

print("\nRegression Equation:")
print(equation)

# === Step 6: Overall F-test ===
print(f"\nF-statistic: {model.fvalue:.4f}")
print(f"F-test p-value: {model.f_pvalue:.6f}")

# === Step 7: Significance test per variable ===
alpha_input = input("\nEnter significance level (default 0.05): ").strip()
alpha = float(alpha_input) if alpha_input else 0.05

print(f"\nVariables significant at alpha = {alpha}:")
for var, pval in model.pvalues.items():
    status = "Significant" if pval < alpha else "Not significant"
    print(f"{var}: p-value = {pval:.6f} → {status}")

# === Step 8: Interpret dummies relative to baseline ===
if baseline_info:
    print("\nInterpretation of dummy variables (relative to baseline):")
    for cat_col, base in baseline_info.items():
        for dcol in [c for c in X.columns if c.startswith(f"{cat_col}_")]:
            level = dcol.split(f"{cat_col}_", 1)[1]
            print(f"- {dcol}: Expected change in y when {cat_col} = '{level}' vs '{base}', holding other predictors constant.")


Dataset:
         y  x1    x2
0   0.2454   1  60.0
1   0.2569   1  60.0
2   0.4984   1  60.0
3   0.2531   2  60.0
4   0.3143   2  60.0
5   0.5316   2  60.0
6   0.5612   3  60.0
7   1.0346   3  60.0
8   1.3985   3  60.0
9   0.2466   1  72.5
10  0.2581   1  72.5
11  0.2566   1  72.5
12  0.2437   2  72.5
13  0.2703   2  72.5
14  0.4845   2  72.5
15  0.3027   3  72.5
16  0.8672   3  72.5
17  1.2242   3  72.5
18  0.2519   1  85.0
19  0.2546   1  85.0
20  0.2496   1  85.0
21  0.2545   2  85.0
22  0.2562   2  85.0
23  0.2904   2  85.0
24  0.2504   3  85.0
25  0.2528   3  85.0
26  0.3007   3  85.0

Regression Equation:
y = 0.9023 + (0.2041)*x1 + (-0.0121)*x2

F-statistic: 9.1367
F-test p-value: 0.001121



Enter significance level (default 0.05):  .05



Variables significant at alpha = 0.05:
const: p-value = 0.021971 → Significant
x1: p-value = 0.002190 → Significant
x2: p-value = 0.017528 → Significant


In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# === Step 1: Load dataset ===
file_path = r"C:\Users\300393449\OneDrive - Douglas College\Documents\4th Semester\2_Business_Statistics_II\Python\DataScratch.xlsx"
sheet_name = "Reg_Model"

df = pd.read_excel(file_path, sheet_name=sheet_name)
print("Dataset:")
print(df)

# =========================
# Helpers
# =========================
def clean_numeric_series(s: pd.Series) -> pd.Series:
    """Strip $, %, commas, spaces; coerce to numeric (NaN on failure)."""
    s = s.astype(str).str.replace(r'[\$,%]', '', regex=True).str.replace(',', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')

def coerce_all_numeric(X: pd.DataFrame) -> pd.DataFrame:
    out = X.copy()
    for c in out.columns:
        if out[c].dtype == 'object' or str(out[c].dtype).startswith('category'):
            out[c] = clean_numeric_series(out[c])
        else:
            out[c] = pd.to_numeric(out[c], errors='coerce')
    return out

def manual_excel_dummy(df_in: pd.DataFrame, cat_col: str, dummy_prefix: str | None = None,
                       categories_order: list[str] | None = None) -> tuple[pd.DataFrame, dict]:
    """
    Create Excel-style dummies for cat_col:
      - If there are C categories, create C-1 dummy cols.
      - The *first* category in categories_order is the baseline (all zeros).
      - Original string column is dropped.
      - Returns (df_out, dummy_info) where dummy_info describes mapping & baseline.

    dummy_info structure:
      {
        'column': cat_col,
        'baseline': <baseline category>,
        'dummies': ['<prefix>_D1', '<prefix>_D2', ...],
        'mapping': { category: [d1,d2,...] }
      }
    """
    d = df_in.copy()

    if cat_col not in d.columns:
        raise ValueError(f"Column '{cat_col}' not found in dataframe.")

    # Clean values to strings for stable matching
    cats = d[cat_col].astype(str).str.strip()
    uniq = cats.unique().tolist()

    # If user didn't supply explicit order, default to alphabetical
    if categories_order:
        order = [c.strip() for c in categories_order]
        missing = [c for c in uniq if c not in order]
        if missing:
            raise ValueError(f"These categories exist in data but not in your order: {missing}")
    else:
        order = sorted(uniq)

    if len(order) < 2:
        raise ValueError(f"Need at least 2 categories in '{cat_col}' to create dummies; found {len(order)}.")

    baseline = order[0]                 # Excel-style: first is baseline
    k = len(order) - 1                  # number of dummy columns
    if dummy_prefix is None:
        dummy_prefix = f"{cat_col}"

    dummy_cols = [f"{dummy_prefix}_D{i+1}" for i in range(k)]

    # Build mapping: category -> dummy vector (length k)
    # baseline (first) is all zeros; category i (1..k) has 1 in position i, zeros elsewhere
    mapping = {}
    for idx, cat in enumerate(order):
        vec = [0]*k
        if idx > 0:
            vec[idx-1] = 1
        mapping[cat] = vec

    # Create the actual dummy columns
    for j in range(k):
        d[dummy_cols[j]] = 0  # initialize

    # Apply mapping row-wise
    for i in range(len(d)):
        cat_val = str(d.loc[i, cat_col]).strip()
        if cat_val not in mapping:
            # unseen category → treat like baseline (all zeros)
            vec = [0]*k
        else:
            vec = mapping[cat_val]
        for j in range(k):
            d.at[i, dummy_cols[j]] = vec[j]

    # Drop original categorical column
    d = d.drop(columns=[cat_col])

    # Prepare info for interpretation / display
    info = {
        'column': cat_col,
        'baseline': baseline,
        'dummies': dummy_cols,
        'mapping': mapping
    }
    return d, info

# =========================
# Step 2: Optional MANUAL (Excel-style) dummy definition
# =========================
use_manual = input("\nUse Excel-style manual dummy setup? (y/N): ").strip().lower() == 'y'
dummy_meta_list = []  # store info for all manually encoded columns

if use_manual:
    print("\nAvailable columns:")
    for c in df.columns:
        print(" -", c)

    while True:
        col_name = input("\nEnter the NAME of the string/categorical column to dummy-encode (or press Enter to stop): ").strip()
        if not col_name:
            break

        # Show distinct categories for convenience
        if col_name in df.columns:
            unique_vals = sorted(df[col_name].astype(str).str.strip().unique().tolist())
            print(f"Distinct values in '{col_name}': {unique_vals}")
        else:
            print(f"[Warning] '{col_name}' not found. Try again.")
            continue

        # Ask user for category order (baseline = first)
        custom_order_in = input(
            "Enter category order separated by commas (first = baseline). "
            "Press Enter to use alphabetical order: "
        ).strip()

        if custom_order_in:
            custom_order = [x.strip() for x in custom_order_in.split(',') if x.strip() != ""]
        else:
            custom_order = None  # fallback to alphabetical

        # Prefix for dummy columns
        prefix_in = input("Enter a prefix for the dummy columns (press Enter to use column name): ").strip()
        prefix = prefix_in if prefix_in else col_name

        # Build manual dummies
        df, dmeta = manual_excel_dummy(df, col_name, dummy_prefix=prefix, categories_order=custom_order)
        dummy_meta_list.append(dmeta)

        # Show mapping like Excel
        print("\n[Manual dummy mapping]")
        print(f"Column: {dmeta['column']}, baseline = '{dmeta['baseline']}', dummy columns = {dmeta['dummies']}")
        for cat, vec in dmeta['mapping'].items():
            print(f"  {cat}: {vec}")

        # Continue encoding more categorical columns?
        more = input("\nAdd another column? (y/N): ").strip().lower()
        if more != 'y':
            break
else:
    print("\nSkipping manual dummy setup. (You can still rely on automatic cleaning and fitting.)")

# =========================
# Step 3: Build model matrices (y and X)
# =========================
if 'y' not in df.columns:
    raise ValueError("Dataset must contain a 'y' column (dependent variable).")

y = df['y']
X = df.drop(columns=['y'])

# =========================
# Step 4: Clean to numeric
# =========================
# Show any non-numeric columns before coercion (for info)
pre_non_num = [c for c in X.columns if (X[c].dtype == 'object' or str(X[c].dtype).startswith('category'))]
if y.dtype == 'object' or str(y.dtype).startswith('category') or pre_non_num:
    print("\n[Info] Coercing non-numeric columns to numeric where possible (stripping $, %, commas).")
    if y.dtype == 'object' or str(y.dtype).startswith('category'):
        print(" - y is non-numeric; cleaning it.")
    if pre_non_num:
        print(" - Non-numeric predictors:", pre_non_num)

y = clean_numeric_series(y)
X = coerce_all_numeric(X)

# Drop rows with missing y or any missing X
mask = y.notna() & X.notna().all(axis=1)
dropped_rows = int(len(y) - mask.sum())
if dropped_rows > 0:
    print(f"\n[Warning] Dropping {dropped_rows} row(s) with non-numeric or missing values after cleaning.")
y = y[mask]
X = X[mask]

# Drop predictors that are constant or entirely NaN
const_or_empty = [c for c in X.columns if X[c].nunique(dropna=True) <= 1]
if const_or_empty:
    print(f"[Info] Dropping constant/empty column(s): {const_or_empty}")
    X = X.drop(columns=const_or_empty)

# Final numeric dtype
X = X.astype(float)
y = y.astype(float)

# Add intercept
X = sm.add_constant(X, has_constant='add')

# =========================
# Step 5: Fit OLS
# =========================
model = sm.OLS(y, X).fit()

# =========================
# Step 6: Regression equation (copy-ready)
# =========================
coef = model.params
eq_parts = [f"y = {coef['const']:.4f}"]
for col in X.columns:
    if col == 'const':
        continue
    eq_parts.append(f"+ ({coef[col]:.4f})*{col}")
equation = " ".join(eq_parts)

print("\nRegression Equation:")
print(equation)

# =========================
# Step 7: Overall F-test
# =========================
print(f"\nF-statistic: {model.fvalue:.4f}")
print(f"F-test p-value: {model.f_pvalue:.6f}")

# =========================
# Step 8: Per-variable significance
# =========================
alpha_input = input("\nEnter significance level (default 0.05): ").strip()
alpha = float(alpha_input) if alpha_input else 0.05

print(f"\nVariables significant at alpha = {alpha}:")
for var, pval in model.pvalues.items():
    status = "Significant" if pval < alpha else "Not significant"
    print(f"{var}: p-value = {pval:.6f} → {status}")

# =========================
# Step 9: Interpret dummy variables (manual Excel-style mapping)
# =========================
if dummy_meta_list:
    print("\nInterpretation of manually created dummy variables (relative to baseline):")
    for meta in dummy_meta_list:
        base = meta['baseline']
        for idx, dcol in enumerate(meta['dummies']):
            # The category corresponding to this dummy is the (idx+1)-th category in the order
            # Recover the level by finding which category has vector with 1 at idx
            target_level = None
            for cat, vec in meta['mapping'].items():
                if idx < len(vec) and vec[idx] == 1:
                    target_level = cat
                    break
            if target_level is None:
                target_level = f"Level_{idx+1}"
            print(f"- {dcol}: Expected change in y when {meta['column']} = '{target_level}' vs '{base}', holding other predictors constant.")


Dataset:
         y  x1    x2
0   0.2454   1  60.0
1   0.2569   1  60.0
2   0.4984   1  60.0
3   0.2531   2  60.0
4   0.3143   2  60.0
5   0.5316   2  60.0
6   0.5612   3  60.0
7   1.0346   3  60.0
8   1.3985   3  60.0
9   0.2466   1  72.5
10  0.2581   1  72.5
11  0.2566   1  72.5
12  0.2437   2  72.5
13  0.2703   2  72.5
14  0.4845   2  72.5
15  0.3027   3  72.5
16  0.8672   3  72.5
17  1.2242   3  72.5
18  0.2519   1  85.0
19  0.2546   1  85.0
20  0.2496   1  85.0
21  0.2545   2  85.0
22  0.2562   2  85.0
23  0.2904   2  85.0
24  0.2504   3  85.0
25  0.2528   3  85.0
26  0.3007   3  85.0



Use Excel-style manual dummy setup? (y/N):  N



Skipping manual dummy setup. (You can still rely on automatic cleaning and fitting.)

Regression Equation:
y = 0.9023 + (0.2041)*x1 + (-0.0121)*x2

F-statistic: 9.1367
F-test p-value: 0.001121



Enter significance level (default 0.05):  .05



Variables significant at alpha = 0.05:
const: p-value = 0.021971 → Significant
x1: p-value = 0.002190 → Significant
x2: p-value = 0.017528 → Significant


In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# === Step 1: Load dataset ===
file_path = r"C:\Users\300393449\OneDrive - Douglas College\Documents\4th Semester\2_Business_Statistics_II\Python\DataScratch.xlsx"
sheet_name = "Reg_Model"

df = pd.read_excel(file_path, sheet_name=sheet_name)
print("Dataset:")
print(df)

# =========================
# Helpers
# =========================
def clean_numeric_series(s: pd.Series) -> pd.Series:
    """Strip $, %, commas, spaces; coerce to numeric (NaN on failure)."""
    s = s.astype(str).str.replace(r'[\$,%]', '', regex=True).str.replace(',', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')

def coerce_all_numeric(X: pd.DataFrame) -> pd.DataFrame:
    out = X.copy()
    for c in out.columns:
        if out[c].dtype == 'object' or str(out[c].dtype).startswith('category'):
            out[c] = clean_numeric_series(out[c])
        else:
            out[c] = pd.to_numeric(out[c], errors='coerce')
    return out

def build_c_minus_1_dummies(df_in: pd.DataFrame, exclude: set[str]) -> tuple[pd.DataFrame, list[dict]]:
    """
    For every object/category column not in `exclude`,
    create (C-1) dummy columns (Excel-style) using an alphabetical baseline.
    Drop original categorical column.
    Return (df_out, meta_list) where meta_list holds mapping/baseline per column.
    """
    d = df_in.copy()
    meta_list = []

    # Identify categorical columns (excluding provided columns)
    cat_cols = [c for c in d.columns
                if c not in exclude and (d[c].dtype == 'object' or str(d[c].dtype).startswith('category'))]

    for col in cat_cols:
        # Clean strings and get unique ordered categories
        col_vals = d[col].astype(str).str.strip()
        uniques = sorted(pd.Index(col_vals.unique()).tolist())
        C = len(uniques)

        if C < 2:
            # Nothing to dummy-encode if only one unique value; just drop it (constant)
            d = d.drop(columns=[col])
            continue

        baseline = uniques[0]               # Excel-style baseline = first alphabetically
        k = C - 1                           # number of dummy columns
        dummy_cols = [f"{col}_D{i+1}" for i in range(k)]

        # Category -> vector mapping (baseline = all zeros; category i has 1 at position i)
        mapping = {}
        for idx, cat in enumerate(uniques):
            vec = [0]*k
            if idx > 0:
                vec[idx-1] = 1
            mapping[cat] = vec

        # Create actual dummy columns
        for j in range(k):
            d[dummy_cols[j]] = 0

        # Assign vectors row-wise
        for i in range(len(d)):
            cat_val = str(d.loc[i, col]).strip()
            vec = mapping.get(cat_val, [0]*k)  # unseen -> baseline
            for j in range(k):
                d.at[i, dummy_cols[j]] = vec[j]

        # Drop original categorical column
        d = d.drop(columns=[col])

        # Save metadata for interpretation
        meta_list.append({
            'column': col,
            'baseline': baseline,
            'dummies': dummy_cols,
            'mapping': mapping
        })

        # Show summary for this column
        print(f"\n[Dummy encoding] '{col}': C={C} → {k} columns, baseline = '{baseline}'")
        for cat in uniques:
            print(f"  {cat}: {mapping[cat]}")

    return d, meta_list

# =========================
# Step 2: Auto (C-1) dummy-encode ALL categorical columns (except 'y')
# =========================
df_enc, dummy_meta_list = build_c_minus_1_dummies(df, exclude={'y'})

# =========================
# Step 3: Build model matrices (y, X)
# =========================
if 'y' not in df_enc.columns:
    raise ValueError("Dataset must contain a 'y' column (dependent variable).")

y = df_enc['y']
X = df_enc.drop(columns=['y'])

# =========================
# Step 4: Clean to numeric, drop bad rows/columns
# =========================
# Inform about non-numeric before coercion
pre_non_num = [c for c in X.columns if (X[c].dtype == 'object' or str(X[c].dtype).startswith('category'))]
if y.dtype == 'object' or str(y.dtype).startswith('category') or pre_non_num:
    print("\n[Info] Coercing non-numeric columns to numeric (stripping $, %, commas).")
    if y.dtype == 'object' or str(y.dtype).startswith('category'):
        print(" - y is non-numeric; cleaning it.")
    if pre_non_num:
        print(" - Non-numeric predictors:", pre_non_num)

y = clean_numeric_series(y)
X = coerce_all_numeric(X)

# Drop rows with missing y or any missing X
mask = y.notna() & X.notna().all(axis=1)
dropped_rows = int(len(y) - mask.sum())
if dropped_rows > 0:
    print(f"\n[Warning] Dropping {dropped_rows} row(s) with non-numeric or missing values after cleaning.")
y = y[mask]
X = X[mask]

# Drop predictors that are constant/empty
const_or_empty = [c for c in X.columns if X[c].nunique(dropna=True) <= 1]
if const_or_empty:
    print(f"[Info] Dropping constant/empty column(s): {const_or_empty}")
    X = X.drop(columns=const_or_empty)

# Final numeric dtype
X = X.astype(float)
y = y.astype(float)

# Add intercept
X = sm.add_constant(X, has_constant='add')

# =========================
# Step 5: Fit OLS
# =========================
model = sm.OLS(y, X).fit()

# =========================
# Step 6: Regression equation (copy-ready)
# =========================
coef = model.params
eq_parts = [f"y = {coef['const']:.4f}"]
for col in X.columns:
    if col == 'const':
        continue
    eq_parts.append(f"+ ({coef[col]:.4f})*{col}")
equation = " ".join(eq_parts)

print("\nRegression Equation:")
print(equation)

# =========================
# Step 7: Overall F-test
# =========================
print(f"\nF-statistic: {model.fvalue:.4f}")
print(f"F-test p-value: {model.f_pvalue:.6f}")

# =========================
# Step 8: Per-variable significance
# =========================
alpha_input = input("\nEnter significance level (default 0.05): ").strip()
alpha = float(alpha_input) if alpha_input else 0.05

print(f"\nVariables significant at alpha = {alpha}:")
for var, pval in model.pvalues.items():
    status = "Significant" if pval < alpha else "Not significant"
    print(f"{var}: p-value = {pval:.6f} → {status}")

# =========================
# Step 9: Interpret dummy variables (relative to baseline)
# =========================
if dummy_meta_list:
    print("\nInterpretation of dummy variables (relative to baseline):")
    for meta in dummy_meta_list:
        base = meta['baseline']
        for idx, dcol in enumerate(meta['dummies']):
            # find which category corresponds to this dummy (vector with 1 at idx)
            level = None
            for cat, vec in meta['mapping'].items():
                if idx < len(vec) and vec[idx] == 1:
                    level = cat
                    break
            if level is None:
                level = f"Level_{idx+1}"
            print(f"- {dcol}: Expected change in y when {meta['column']} = '{level}' vs '{base}', holding other predictors constant.")


Dataset:
         y  x1    x2
0   0.2454   1  60.0
1   0.2569   1  60.0
2   0.4984   1  60.0
3   0.2531   2  60.0
4   0.3143   2  60.0
5   0.5316   2  60.0
6   0.5612   3  60.0
7   1.0346   3  60.0
8   1.3985   3  60.0
9   0.2466   1  72.5
10  0.2581   1  72.5
11  0.2566   1  72.5
12  0.2437   2  72.5
13  0.2703   2  72.5
14  0.4845   2  72.5
15  0.3027   3  72.5
16  0.8672   3  72.5
17  1.2242   3  72.5
18  0.2519   1  85.0
19  0.2546   1  85.0
20  0.2496   1  85.0
21  0.2545   2  85.0
22  0.2562   2  85.0
23  0.2904   2  85.0
24  0.2504   3  85.0
25  0.2528   3  85.0
26  0.3007   3  85.0

Regression Equation:
y = 0.9023 + (0.2041)*x1 + (-0.0121)*x2

F-statistic: 9.1367
F-test p-value: 0.001121



Enter significance level (default 0.05):  .05



Variables significant at alpha = 0.05:
const: p-value = 0.021971 → Significant
x1: p-value = 0.002190 → Significant
x2: p-value = 0.017528 → Significant
